In [1]:
#highlighting favorite destination
# connect to sql
#web

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time 
import sqlite3



In [2]:
url = "https://www.gulliver.co.il/Content/SalePage.aspx?pageId=264&module=Flights"
r = requests.get(url)
c = r.content
soup=BeautifulSoup(c,"html.parser")
all = soup.find_all("ul")




In [3]:
url2 = 'https://www.gulliver.co.il/Content/SalePage.aspx?pageId=99&module=Flights'
r2 = requests.get(url2)
c2 = r2.content
soup2=BeautifulSoup(c2,"html.parser")
all2 = soup2.find_all("ul")  

In [31]:
# mivzaim
sale = []


for i in range(len(soup.find_all("strong",{'style':'font-size:1em;'}))):
    m={}
    m['destination'] = soup.find_all("strong",{'style':'font-size:1em;'})[i].text.replace('\t', '')
    m['company'] = soup.find_all("span", {'class':'LastDeals_hotel'})[i].text.replace('\t', '')
    m['vacationDate'] = soup.find_all("span", {'class':'LastDeals_Date'})[i].text.replace('\t', '')
    m['PriceInDollars'] = int(soup.find_all("strong", {'class':'LastDeals_price'})[i].text.replace('$',''))
    m['offerDate'] = time.strftime("%d/%m/%Y")
    sale.append(m)

In [32]:
#last minute

last = []



for i in range(len(all2)):
    m={}
    m['destination'] = soup2.find_all("strong",{'style':'font-size:1em;'})[i].text.replace('\t', '').split()[0]
    m['company'] = 'Last Minute'
    m['vacationDate'] = soup2.find_all("span", {'class':'LastDeals_Details'})[i].text.replace('מחיר לאדםתאריכי טיסה','') 
    m['PriceInDollars'] = int(soup2.find_all("strong", {'class':'LastDeals_price'})[i].text.replace('$','').replace('€',''))
    m['offerDate'] = time.strftime("%d/%m/%Y")
    last.append(m)

In [117]:
df=pd.DataFrame(sale)
cols = df.columns.tolist()
cols = [cols[3] , cols[2] , cols[1],cols[4],cols[0]]
df = df[cols] 


In [116]:
df2 = pd.DataFrame(last)
df2 = df2[cols]


In [118]:
def highlight_cols(s):
    
    if s < 100 :
        color = 'grey' 
        
    elif s < 180 :
        color = 'green'
        
    else :
        color = None
        
    return 'background-color: %s' % color

s=df.style.applymap(highlight_cols, subset=pd.IndexSlice[:, [ 'PriceInDollars']])





In [115]:
merged = df2.append(df,ignore_index=True)


In [102]:
# sql 


def create ():
    conn=sqlite3.connect("flights.db")
    cur=conn.cursor()
    cur.execute("CREATE TABLE IF NOT EXISTS flights (id INTEGER PRIMARY KEY,offerDate date,destination TEXT,company TEXT,vacationDate date,PriceInDollars INTEGER)")
    conn.commit()
    conn.close()
          
    
def gettable():
    conn = sqlite3.connect("flights.db")
    table = pd.read_sql_query("SELECT * FROM flightData", conn)
    return table
    conn.close()        
    

def drop_table():
    conn = sqlite3.connect("flights.db")
    cur = conn.cursor()
    cur.execute("DROP TABLE flightData")
    conn.commit()
    conn.close()
    
def insert(merged) :  
    conn = sqlite3.connect("flights.db")   
    df = merged
    df.to_sql("flightData", conn, if_exists='append')
    conn.commit()
    conn.close()

    
def findLastDate():
    conn=sqlite3.connect("flights.db") 
    cur = conn.cursor()
    lastDate = cur.execute("SELECT distinct max(offerDate) FROM flightData").fetchone()[0]
    conn.close()
    return lastDate


In [114]:
#drop_table()
#create()


if findLastDate()  < max(merged.offerDate) :
    insert(merged)
else :
    print ("Already in database")


Already in database


In [113]:

t = gettable()
t

,index,offerDate,destination,company,vacationDate,PriceInDollars
0,0,24/06/2018,אנטליה,Last Minute,28.06 - 24.06,132
1,1,24/06/2018,רודוס,Last Minute,02.07 - 25.06,161
2,2,24/06/2018,כרתים,Last Minute,01.07 - 25.06,161
3,3,24/06/2018,לרנקה,Last Minute,04.07 - 25.06,164
4,4,24/06/2018,קטניה,Last Minute,04.07 - 27.06,151
5,5,24/06/2018,ורנה,Last Minute,02.07 - 25.06,179
6,6,24/06/2018,ורונה,Last Minute,04.07 - 27.06,179
7,7,24/06/2018,קוס,Last Minute,29.06 - 25.06,180
8,8,24/06/2018,בלגרד,Last Minute,29.06 - 25.06,187
9,9,24/06/2018,בוקרשט,Last Minute,10.07 - 25.06,191
